In [1]:
print("ok")

ok


In [2]:
pip install torch

   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   ----- ---------------------------------- 0.8/6.2 MB 1.0 MB/s eta 0:00:06
   -------- ------------------------------- 1.3/6.2 MB 1.3 MB/s eta 0:00:04
   ------------- -------------------------- 2.1/6.2 MB 1.4 MB/s eta 0:00:03
   ---------------- ----------------------- 2.6/6.2 MB 1.4 MB/s eta 0:00:03
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2

## Data Preparation
The dataset contains around 1000 images of hairs in the separate folders for training and test sets.

## Reproducibility
Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs.

Therefore, in this homework we suggest to set the random number seed generators by:

````
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
````
Also, use PyTorch of version 2.8.0 (that's the one in Colab).

## Model
For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

You need to develop the model with following structure:

- The shape for input should be (3, 200, 200) (channels first format in PyTorch)
- Next, create a convolutional layer (nn.Conv2d):
  - Use 32 filters (output channels)
  - Kernel size should be (3, 3) (that's the size of the filter)
  - Use 'relu' as activation
- Reduce the size of the feature map with max pooling (nn.MaxPool2d)
  - Set the pooling size to (2, 2)
- Turn the multi-dimensional result into vectors using flatten or view
- Next, add a nn.Linear layer with 64 neurons and 'relu' activation
- Finally, create the nn.Linear layer with 1 neuron - this will be the output
  - The output layer should have an activation - use the appropriate activation for the binary classification case
As optimizer use torch.optim.SGD with the following parameters:

````torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)````

In [3]:
import requests
import zipfile
import os

url = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"
Data = "data.zip"

In [4]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    """
    A Convolutional Neural Network model for binary classification 
    with a single output neuron, designed according to the specifications.
    Input shape: (3, 200, 200)
    """
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # 1. Convolutional Layer (nn.Conv2d)
        # Input: (3, 200, 200)
        # Output: (32, 198, 198)  -> Calculated as: 200 - 3 + 1 = 198
        self.conv1 = nn.Conv2d(
            in_channels=3,         # Input channels (RGB image)
            out_channels=32,       # Number of filters
            kernel_size=(3, 3)     # Filter size
        )
        
        # 2. Max Pooling Layer (nn.MaxPool2d)
        # Input: (32, 198, 198)
        # Output: (32, 99, 99)   -> Calculated as: 198 / 2 = 99
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        
        # Calculate the size of the feature vector after Conv and Pooling
        # H_out = (200 - 3 + 1) / 2 = 99
        # W_out = (200 - 3 + 1) / 2 = 99
        # Number of features = Channels * H_out * W_out = 32 * 99 * 99 = 313632
        
        # 3. Fully Connected Layer 1 (nn.Linear)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        
        # 4. Fully Connected Layer 2 (nn.Linear) - Output layer
        # Output: 1 neuron (logit for binary classification)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # 1. Conv Layer + ReLU activation
        x = self.conv1(x)
        x = F.relu(x)
        
        # 2. Max Pooling
        x = self.pool(x)
        
        # 3. Flatten/View - Convert multi-dimensional result to a vector
        # '-1' lets PyTorch infer the batch size dimension
        x = x.view(x.size(0), -1) 
        
        # 4. Linear Layer 1 + ReLU activation
        x = self.fc1(x)
        x = F.relu(x)
        
        # 5. Output Linear Layer 
        # NOTE: We do NOT apply a Sigmoid here because we selected 
        # nn.BCEWithLogitsLoss() which handles the Sigmoid internally.
        x = self.fc2(x)
        
        return x

# --- Model Instantiation and Optimizer Setup ---

# Instantiate the model
model = CNNModel()

# Define the optimizer as specified
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# Print the model structure
print(model)

# Print the optimizer
print(optimizer)

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.002
    maximize: False
    momentum: 0.8
    nesterov: False
    weight_decay: 0
)


## Question 1
Which loss function you will use?

- nn.MSELoss()
- nn.BCEWithLogitsLoss()
- nn.CrossEntropyLoss()
- nn.CosineEmbeddingLoss()

## Question 2
What's the total number of parameters of the model? You can use torchsummary or count manually.

In PyTorch, you can find the total number of parameters using:

````
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
`````

In [6]:
pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl.metadata (296 bytes)
Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


## Generators and Training
For the next two questions, use the following transformation for both train and test sets:

````
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])
````
- We don't need to do any additional pre-processing for the images.
- Use batch_size=20
- Use shuffle=True for both training, but False for test.
Now fit the model.

In [8]:
conda install torchvision -c pytorch

Jupyter detected...
3 channel Terms of Service accepted
Channels:
 - pytorch
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [9]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [19]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import os

# --- A. Define Device and Criterion ---

# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the Criterion (Loss Function)
criterion = nn.BCEWithLogitsLoss()

# --- B. Define Transforms ---

# Transformations required by the homework
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

# --- C. Load Data, Split, and Create Loaders ---

# 🛑 CRITICAL FIX: The path is corrected to account for nested extraction
data_dir = './data/data' 

# 1. Load the entire training dataset
# This should now correctly find the images inside './data/data/train'
full_train_data = ImageFolder(
    root=os.path.join(data_dir, 'train'),
    transform=train_transforms
)

# 2. Define Split Sizes (80% train, 20% validation)
total_size = len(full_train_data)
TRAIN_SIZE = int(0.8 * total_size)
VAL_SIZE = total_size - TRAIN_SIZE 

# Set the random seed for reproducible split results
SEED = 42
torch.manual_seed(SEED) 
train_dataset, validation_dataset = random_split(full_train_data, [TRAIN_SIZE, VAL_SIZE])


# 3. Create DataLoaders
BATCH_SIZE = 20 # Required by homework
# Using half of available CPU cores for efficient loading
num_workers = os.cpu_count() // 2 or 1 

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, # Required: shuffle=True for training
    num_workers=num_workers
)
validation_loader = DataLoader(
    validation_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, # Required: shuffle=False for validation
    num_workers=num_workers
)

print(f"Total images found: {total_size}")
print(f"Train samples: {len(train_dataset)}, Validation samples: {len(validation_dataset)}")

Using device: cpu
Total images found: 800
Train samples: 640, Validation samples: 160


In [20]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6510, Acc: 0.6406, Val Loss: 0.6394, Val Acc: 0.6687
Epoch 2/10, Loss: 0.5979, Acc: 0.6719, Val Loss: 0.6560, Val Acc: 0.5938
Epoch 3/10, Loss: 0.5448, Acc: 0.7016, Val Loss: 0.6432, Val Acc: 0.7000
Epoch 4/10, Loss: 0.5003, Acc: 0.7438, Val Loss: 0.6291, Val Acc: 0.6875
Epoch 5/10, Loss: 0.4526, Acc: 0.7844, Val Loss: 0.6098, Val Acc: 0.6937
Epoch 6/10, Loss: 0.4173, Acc: 0.7953, Val Loss: 0.5565, Val Acc: 0.7250
Epoch 7/10, Loss: 0.3933, Acc: 0.8219, Val Loss: 0.5695, Val Acc: 0.7312
Epoch 8/10, Loss: 0.2936, Acc: 0.8906, Val Loss: 0.5860, Val Acc: 0.6937
Epoch 9/10, Loss: 0.2228, Acc: 0.9234, Val Loss: 0.9849, Val Acc: 0.6750
Epoch 10/10, Loss: 0.2491, Acc: 0.8844, Val Loss: 0.6474, Val Acc: 0.7562


## Question 3
What is the median of training accuracy for all the epochs for this model?

- 0.05
- 0.12
- 0.40
- **0.84**

## Question 4
What is the standard deviation of training loss for all the epochs for this model?

- 0.007
- 0.078
- **0.171**
- 1.710

In [21]:
import numpy as np

# Data from the Training Log 

# Training Accuracy values (Acc)
acc_values = np.array([
    0.6406, 0.6719, 0.7016, 0.7438, 0.7844,
    0.7953, 0.8219, 0.8906, 0.9234, 0.8844
])

# Training Loss values (Loss)
loss_values = np.array([
    0.6510, 0.5979, 0.5448, 0.5003, 0.4526,
    0.4173, 0.3933, 0.2936, 0.2228, 0.2491
])

median_accuracy = np.median(acc_values)

print("--- Question 3 Code Result ---")
print(f"Median Training Accuracy: {median_accuracy:.4f}")

--- Question 3 Code Result ---
Median Training Accuracy: 0.7898


In [22]:
std_dev_loss = np.std(loss_values, ddof=1)

print("\n--- Question 4 Code Result ---")
print(f"Standard Deviation of Training Loss (Sample): {std_dev_loss:.4f}")


--- Question 4 Code Result ---
Standard Deviation of Training Loss (Sample): 0.1458


## Data Augmentation
For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:
````
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),
````

In [23]:
from torchvision import transforms

# Define the training transformations with augmentations
train_transforms = transforms.Compose([
    # 1. Augmentations
    transforms.RandomRotation(50), 
    transforms.RandomResizedCrop(
        200,                # Output size: 200x200
        scale=(0.9, 1.0),   # Crop area scale
        ratio=(0.9, 1.1)    # Aspect ratio perturbation
    ),
    transforms.RandomHorizontalFlip(),
    
    # 2. Required Core Transformations
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

# For the validation/test set, we only need the standard transformations (no augmentation)
validation_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Training transforms updated successfully with augmentations.")

Training transforms updated successfully with augmentations.


In [24]:
import torch
import torch.nn as nn
# Note: transforms, ImageFolder, DataLoader, random_split, and os are assumed imported from previous cells

# --- A. Define Device and Criterion (If not defined yet) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss()

# --- B. Define Transforms (Copied from your last successful run) ---

# Augmented transforms for the training set
train_transforms = transforms.Compose([
    transforms.RandomRotation(50), 
    transforms.RandomResizedCrop(
        200,                # Output size: 200x200
        scale=(0.9, 1.0),   # Crop area scale
        ratio=(0.9, 1.1)    # Aspect ratio perturbation
    ),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) 
])

# Non-augmented transforms for the validation set
validation_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# C. Load Data, Split, and Create Loaders 

#CRITICAL PATH: The path corrected earlier to handle nested extraction
data_dir = './data/data' 

# 1. Load the entire training dataset, applying the AUGMENTED transforms
full_train_data = ImageFolder(
    root=os.path.join(data_dir, 'train'),
    transform=train_transforms # 👈 Using AUGMENTED transforms
)

# 2. Define Split Sizes
total_size = len(full_train_data)
TRAIN_SIZE = int(0.8 * total_size)
VAL_SIZE = total_size - TRAIN_SIZE 

# Split the data
SEED = 42
torch.manual_seed(SEED) 
# Note: random_split requires the base dataset to be defined first.
# We then apply the NON-AUGMENTED transform to the validation subset implicitly
# when loading it later, but here we define the datasets for loading:
train_split, val_split = random_split(full_train_data, [TRAIN_SIZE, VAL_SIZE])


# --- Create DataLoaders ---
BATCH_SIZE = 20
num_workers = os.cpu_count() // 2 or 1 

# DataLoader for Training (Augmented)
train_loader = DataLoader(
    train_split, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=num_workers
)

# DataLoader for Validation (Using the split portion of the augmented dataset)
validation_loader = DataLoader(
    val_split, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=num_workers
)

print(f"Total images found: {total_size}")
print(f"Train samples (Augmented): {len(train_split)}, Validation samples (Augmented): {len(val_split)}")

Total images found: 800
Train samples (Augmented): 640, Validation samples (Augmented): 160


In [25]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6289, Acc: 0.6781, Val Loss: 0.6434, Val Acc: 0.6625
Epoch 2/10, Loss: 0.6158, Acc: 0.6484, Val Loss: 0.9360, Val Acc: 0.5875
Epoch 3/10, Loss: 0.6148, Acc: 0.6266, Val Loss: 0.6692, Val Acc: 0.6250
Epoch 4/10, Loss: 0.6034, Acc: 0.6562, Val Loss: 0.6514, Val Acc: 0.6750
Epoch 5/10, Loss: 0.5580, Acc: 0.6984, Val Loss: 0.5714, Val Acc: 0.6937
Epoch 6/10, Loss: 0.5493, Acc: 0.6906, Val Loss: 0.5790, Val Acc: 0.6813
Epoch 7/10, Loss: 0.5227, Acc: 0.7141, Val Loss: 0.5592, Val Acc: 0.7000
Epoch 8/10, Loss: 0.5268, Acc: 0.7312, Val Loss: 0.5337, Val Acc: 0.6813
Epoch 9/10, Loss: 0.5143, Acc: 0.7141, Val Loss: 0.6140, Val Acc: 0.7063
Epoch 10/10, Loss: 0.5337, Acc: 0.7234, Val Loss: 0.5967, Val Acc: 0.6562


## Question 5
Let's train our model for 10 more epochs using the same code as previously.

Note: make sure you don't re-create the model. we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.008
- 0.08
- **0.88**
- 8.88

## Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- 0.08
- **0.28**
- 0.68
- 0.98

In [26]:
import numpy as np

#Data from the Training Log (With Augmentations)

# Training Accuracy values (Acc)
acc_values_aug = np.array([
    0.6781, 0.6484, 0.6266, 0.6562, 0.6984,
    0.6906, 0.7141, 0.7312, 0.7141, 0.7234
])

# Validation Loss values (Val Loss)
loss_values_aug = np.array([
    0.6434, 0.9360, 0.6692, 0.6514, 0.5714,
    0.5790, 0.5592, 0.5337, 0.6140, 0.5967
])

#Question 5: Median Training Accuracy 

# Calculate the median of the augmented accuracy values
median_accuracy_aug = np.median(acc_values_aug)

print("--- Question 5 Code Result ---")
print(f"Median Training Accuracy (Augmented): {median_accuracy_aug:.4f}")

--- Question 5 Code Result ---
Median Training Accuracy (Augmented): 0.6945


In [27]:
#Question 6: Standard Deviation of Validation Loss

# Calculate the sample standard deviation (ddof=1) of the augmented validation loss values
std_dev_loss_aug = np.std(loss_values_aug, ddof=1)

print("\n--- Question 6 Code Result ---")
print(f"Standard Deviation of Validation Loss (Augmented): {std_dev_loss_aug:.4f}")


--- Question 6 Code Result ---
Standard Deviation of Validation Loss (Augmented): 0.1141
